### **Cas Kaggle** - Àngela Obón Soto (1600359) i Muniba Liaqat Ali Ali (1635042)

https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data

El dataset que analitzarem conté informació sobre l'activitat dels llistats d'Airbnb a la ciutat de Nova York durant el 2019. Aquest conjunt de dades inclou detalls sobre els amfitrions, la ubicació geogràfica de les propietats, preus, disponibilitat, ressenyes i més.
**L'Objectiu Principal** d'aquesta anàlisi és explorar i comprendre els factors que influeixen en els preus i la disponibilitat de les llistes a Nova York, així com identificar patrons geogràfics i mètriques relacionades amb l'activitat dels amfitrions; i proporcionar recomanacions basades en l'anàlisi de dades.". En particular, busquem respondre les preguntes següents. 

1. Quines diferències hi ha en els preus entre les diferents àrees de la ciutat?
2. Quins factors tenen més influència en el preu dels llistats?
3. Quines àrees tenen la major disponibilitat i per què?
4. És possible identificar patrons a l'activitat dels amfitrions més ocupats?
5. Quins insights geoespacials es poden derivar de l'anàlisi de les dades?
6. Quins patrons s'observen a la distribució de preus entre diferents barris i tipus d'habitacions?
7. Quins són els barris més populars o més rendibles?
8. Quins factors tenen un impacte significatiu en la fixació de preus?

Aquesta anàlisi no només permet entendre les dinàmiques del mercat d'Airbnb a Nova York, sinó que també proporciona informació útil tant per a amfitrions que busquen optimitzar els llistats com per a usuaris interessats a trobar les millors opcions d'allotjament.

In [17]:
#imports

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.preprocessing import TargetEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
#exploracion inicial, veiem les dades del dataset

df = pd.read_csv('AB_NYC_2019.csv')
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


#### **Descripció de Variables Clau**
**price:** Preu de l'allotjament (variable objectiu en moltes anàlisis).
Rang observat: 0 a 10,000 (abans del filtratge d'outliers).

**room_type:** Tipus d'habitació (categoria: Entire home/apt, Private room, Shared room).

**neighbourhood_group:** Agrupació geogràfica dels barris (categories: Bronx, Brooklyn, Manhattan, Queens, Staten Island).

**availability_365:** dies de disponibilitat de l'allotjament durant l'any.

**number_of_reviews:** Nombre total de ressenyes rebudes per cada llistat.

**minimum_nights (Nits mínimes):** Determina les restriccions de reserva i pot influir en la disponibilitat i popularitat dels llistats.

In [19]:
# Resumen extenso de las variables del dataset

def summarize_dataset(df):
    print("======================== ESTRUCTURA DEL DATASET ========================")
    print(df.info())
    
    print("\n======================== COLUMNAS EN EL DATASET ========================")
    print(df.columns.to_list())
    
    print("\n======================== TAMAÑO DEL DATASET ============================")
    print(f"Número de filas: {df.shape[0]}, Número de columnas: {df.shape[1]}")
    
    print("\n======================== PRIMERAS LÍNEAS DEL DATASET ===================")
    print(df.head())
    
    print("\n======================== VALORES NULOS ================================")
    nans = (df.isna().sum().sort_values() / len(df) * 100).round(2)
    print(nans)
    
    print("\n======================== TIPO DE DATOS ================================")
    print(df.dtypes)
    
    print("\n======================== DESCRIPCIÓN GENERAL ===========================")
    print(df.describe(include='all'))
    
    
    
    
    #return variables_summary_df

# Para usar la función, simplemente llamarla con el DataFrame:
# resumen_df = summarize_dataset(df)


In [20]:
resumen_df = summarize_dataset(df)

======================== ESTRUCTURA DEL DATASET ========================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews   

In [21]:

print("\n======================== RESUMEN DE VARIABLES =========================")
variables_summary = {
    "Column": df.columns,
    "Data Type": df.dtypes,
    "Null Values (%)": (df.isnull().sum() / len(df)) * 100,
    "Unique Values": df.nunique()
}

variables_summary_df = pd.DataFrame(variables_summary).reset_index(drop=True)

variables_summary_df



======================== RESUMEN DE VARIABLES =========================


,Column,Data Type,Null Values (%),Unique Values
0,id,int64,0.000000,48895
1,name,object,0.032723,47905
2,host_id,int64,0.000000,37457
3,host_name,object,0.042949,11452
4,neighbourhood_group,object,0.000000,5
5,neighbourhood,object,0.000000,221
6,latitude,float64,0.000000,19048
7,longitude,float64,0.000000,14718
8,room_type,object,0.000000,3
9,price,int64,0.000000,674


### Objectiu del Dataset

**L'Objectiu Principal** d'aquesta anàlisi és explorar i comprendre els factors que influeixen en els preus i la disponibilitat de les llistes d'Airbnb a Nova York. A més, es busca identificar patrons geogràfics i mètriques relacionades amb l'activitat dels amfitrions, per tal de proporcionar recomanacions basades en l'anàlisi de dades. Entre els punts específics a abordar, destaquen:

1. Identificar els factors que més influeixen en els preus (tipus d'habitació, localització, ressenyes, etc.).
2. Analitzar patrons de disponibilitat i activitat segons zones i amfitrions.
3. Preservar la qualitat del dataset per assegurar conclusions representatives i útils.

### Justificació de la Neteja de Dades

Per assolir aquests objectius, és fonamental tenir un dataset net i representatiu. Això implica tractar duplicats, valors extrems i valors nuls, assegurant que les dades reflecteixin amb precisió el mercat i evitant biaixos en els resultats.

---

### Neteja de Dades: Duplicats, Valors Extrems i Tractament de NaNs

#### Verificació de Duplicats
- **Motiu**: Les files duplicades poden distorsionar l'anàlisi i fer que els resultats siguin menys fiables.
- **Acció**: Verifiquem duplicats en el dataset.
- **Resultat**: No detectem duplicats (`duplicates_count = 0`), la qual cosa garanteix que cada observació és única i vàlida per a l'anàlisi.

---

#### Filtrat de Valors Extrems

1. **Preus**:
   - **Motiu**: Els valors extrems poden distorsionar les mètriques i afectar els models predictius.
   - **Acció**: Eliminem registres amb:
     - `price = 0` (inconsistents amb l'objectiu d'un lloguer de pagament).
     - `price > 1000` (listats de luxe extrems que no representen el mercat general).
   - **Resultat**: Assegurem que els preus estiguin en el rang (0, 1000]

2. **Nits Mínimes**:
   - **Motiu**: Valors molt alts (>30 dies) no reflecteixen lloguers a curt termini, el focus principal d'Airbnb.
   - **Acció**: Eliminem registres amb `minimum_nights > 30`.
   - **Resultat**: Preservem els llistats rellevants per a l'objectiu principal d'Airbnb.

---

#### Tractament de NaNs

1. **Motiu**:
   - Eliminar valors nuls pot reduir significativament la mida del dataset i generar biaixos, ja que molts NaNs tenen significat (e.g., llistats sense ressenyes).
   - Alguns camps com `last_review` i `reviews_per_month` són rellevants només per a llistats amb activitat. Els NaNs no són errors, sinó indicadors de manca d'activitat en aquests casos.

2. **Accions**:
   - **Valors descriptius**: Omplim els NaNs de `name` i `host_name` amb valors genèrics (`No Name`, `Unknown`) per preservar la usabilitat.
   - **Ressenyes**:
     - `last_review`: Convertim a format de data amb NaNs gestionats com `NaT`.
     - `reviews_per_month`: Omplim els NaNs amb `0`, indicant manca d'activitat de ressenyes.
   - Això assegura que les dades siguin coherents i flexibles per a l'anàlisi.

3. **Resultat**:
   - Preservem la flexibilitat del dataset per respondre preguntes específiques.
   - Mantenim la representativitat, evitant esbiaixos per l’eliminació de llistats sense ressenyes (per exemple, nous llistats o zones emergents).

---

### Impacte de la Neteja

1. **Duplicats**: No eliminem files, ja que no hi ha duplicats.
2. **Valors Extrems**: Garantim que el dataset sigui més representatiu i coherent amb el mercat de lloguers de curt termini.
3. **NaNs**: ransformem aquests valors en significatius, preservant la informació i assegurant que el dataset sigui complet i fiable per a l’anàlisi.


In [22]:
#Identificar columnes amb valors nuls 

null_columns = df.columns[df.isnull().any()]
null_columns
print(df[null_columns].dtypes)


name                  object
host_name             object
last_review           object
reviews_per_month    float64
dtype: object


In [23]:
# Maneig inicial de valors nuls:
# - 'name' i 'host_name': omplir amb un valor genéric
df['name'] = df['name'].fillna('No Name')
df['host_name'] = df['host_name'].fillna('Unknown')
# - 'last_review': Convertir a data, nuls es mantenen com a NaT
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')
# - 'reviews_per_month': Omplir nuls amb 0 (llistats sense ressenyes actives)
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)

In [24]:
# Verificar duplicats
duplicates_count = df.duplicated().sum()

In [25]:
# Outliers preliminars: identificació en preus i nits minimes
# Definimos límits
df = df[(df['price'] > 0) & (df['price'] <= 1000)]  # Filtrar preuss extrems
df = df[df['minimum_nights'] <= 30]  # Filtrar nits mínimas fora de rang lógic

In [26]:
# Resum final
cleaned_shape = df.shape
# Mostrar resultats
print("\nColumnas amb valor nul:")
print(null_columns)

# Identificar columnas con valores nulos després de la limpieza
null_columns_after = df.columns[df.isnull().any()]
print("\nColumnas amb valors nuls després del tractament:")
print(null_columns_after)


print(f"\nNúmero de duplicats trobat: {duplicates_count}")

print(f"\nDimensions del dataset despres de la neteja: {cleaned_shape}")




Columnas amb valor nul:
Index(['name', 'host_name', 'last_review', 'reviews_per_month'], dtype='object')

Columnas amb valors nuls després del tractament:
Index(['last_review'], dtype='object')

Número de duplicats trobat: 0

Dimensions del dataset despres de la neteja: (47924, 16)


El resultat indica que després del tractament inicial, la columna last_review encara conté valors nuls. Això és completament esperable per les raons següents:

En convertir last_review a format de data amb pd.to_datetime, els valors que originalment eren nuls s'han mantingut com a NaT (Not a Time), que és l'equivalent de "nul" per a dades de tipus datetime.
Això és lògic, ja que no hi ha informació sobre la darrera ressenya per als llistats que mai no han rebut ressenyes.

El dataset net té ara 47,924 filas i 16 columnes.

#### **Resum estadístic**

El resum estadístic és crucial perquè ens permet entendre ràpidament les característiques generals del dataset, com ara els valors promig, la dispersió, els valors extrems i la distribució de les variables. 

In [28]:
# Calcular estadísticas específicas
resumen_precio = {
    "Media Precio (USD)": df['price'].mean(),
    "Mínimo Precio (USD)": df['price'].min(),
    "Máximo Precio (USD)": df['price'].max()
}

resumen_minimum_nights = {
    "Media Noches Mínimas": df['minimum_nights'].mean(),
    "Máximo Noches Mínimas": df['minimum_nights'].max()
}

resumen_disponibilidad = {
    "Media Disponibilidad (días)": df['availability_365'].mean()
}

# Mostrar resultados
print("Resumen de Precios:", resumen_precio)
print("Resumen de Noches Mínimas:", resumen_minimum_nights)
print("Resumen de Disponibilidad Anual:", resumen_disponibilidad)

Resumen de Precios: {'Media Precio (USD)': np.float64(141.31483181704365), 'Mínimo Precio (USD)': np.int64(10), 'Máximo Precio (USD)': np.int64(1000)}
Resumen de Noches Mínimas: {'Media Noches Mínimas': np.float64(5.576016192304482), 'Máximo Noches Mínimas': np.int64(30)}
Resumen de Disponibilidad Anual: {'Media Disponibilidad (días)': np.float64(111.43414573074034)}


Després de realitzar una neteja al nostre dataset, veurem la relació que hi ha entre els seus atributs. 

In [27]:
# Relacions entre atributs
# Correlacions entre variables numèriques

correlation_matrix = df.corr()

ValueError: could not convert string to float: 'Brooklyn'